In [1]:
import os
import pandas as pd
import pyodbc
import openai
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain import SQLDatabaseChain
from langchain.chains import SQLDatabaseSequentialChain
from langchain.agents import AgentExecutor

#from common.prompts import MSSQL_PROMPT, MSSQL_AGENT_PREFIX, MSSQL_AGENT_FORMAT_INSTRUCTIONS

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))
    
from langchain.prompts import PromptTemplate

In [2]:
#from common.prompts import MSSQL_PROMPT, MSSQL_AGENT_PREFIX, MSSQL_AGENT_FORMAT_INSTRUCTIONS

In [3]:
api_type = "azure"
api_base_url = "https://openaidemonubiral.openai.azure.com/"
api_version = "2023-03-15-preview"
azure_api_key = "ff5c606c134e4d1dae3426a412df834a"

openai.api_type = api_type
#openai.api_base = api_base_url
openai.api_version = api_version
#openai.api_key = azure_api_key

os.environ["OPENAI_API_BASE"] = api_base_url
os.environ["OPENAI_API_KEY"] = azure_api_key

In [4]:
BASE_URL = "https://openaidemonubiral.openai.azure.com/"
API_KEY = "ff5c606c134e4d1dae3426a412df834a"
DEPLOYMENT_NAME = "nubiral-lab-01"


In [5]:
#sudo -H pip install pyodbc

In [6]:
import pyodbc
server = 'sql-serveropenai.database.windows.net'
database = 'database-demo'
username = 'admindemo'
password = 'nubiral2023!'   
driver= '{ODBC Driver 18 for SQL Server}'

In [7]:
conn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

In [8]:
cursor.execute("SELECT TOP 5 * FROM customer_shopping ORDER BY invoice_date DESC") 
row = cursor.fetchone() 
while row:
    print (row) 
    row = cursor.fetchone()

('I102162', 'C274417', 'Female', 64, 'Food & Beverage', 3, 1569.0, 'Credit Card', datetime.date(2023, 12, 2), 'Metropol AVM')
('I110730', 'C267745', 'Female', 28, 'Food & Beverage', 5, 2615.0, 'Cash', datetime.date(2023, 12, 2), 'Istinye Park')
('I116593', 'C207927', 'Female', 27, 'Souvenir', 3, 3519.0, 'Debit Card', datetime.date(2023, 12, 2), 'Mall of Istanbul')
('I123893', 'C604997', 'Female', 65, 'Clothing', 3, 90024.0, 'Cash', datetime.date(2023, 12, 2), 'Emaar Square Mall')
('I124354', 'C260649', 'Male', 59, 'Clothing', 1, 30008.0, 'Cash', datetime.date(2023, 12, 2), 'Cevahir AVM')


In [9]:
db_url = 'DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password

In [10]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

In [11]:
db_config = {   'drivername': 'mssql+pyodbc',
                'username': username +'@'+ server,
                'password': password,
                'host': server,
                'port': 1433,
                'database': database,
                'query': {'driver': 'ODBC Driver 18 for SQL Server'}
            }

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

## Query with LLM

In [12]:
MSSQL_PROMPT = """
You are an MS SQL expert. Given an input question, first create a syntactically correct MS SQL query to run, then look at the results of the query and return the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the TOP clause as per MS SQL. You can order the results to return the most informative data in the database.

Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in square brackets ([]) to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

**Do not use double quotes on the SQL query**. 

Your response should be in Markdown. Answer the question in spanish.

** ALWAYS before giving the Final Answer, try another method**. Then reflect on the answers of the two methods you did and ask yourself if it answers correctly the original question. If you are not sure, try another method.
If the runs does not give the same result, reflect and try again two more times until you have two runs that have the same result. If you still cannot arrive to a consistent result, say that you are not sure of the answer. But, if you are sure of the correct answer, create a beautiful and thorough response. DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE. 

ALWAYS, as part of your final answer, explain how you got to the answer on a section that starts with: \n\nExplanation:\n. Include the SQL query as part of the explanation section.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here
Explanation:

For example:
<=== Beginning of example

Question: How many people died of covid in Texas in 2020?
SQLQuery: SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'
SQLResult: [(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Answer: There were 27437 people who died of covid in Texas in 2020.


Explanation:
I queried the covidtracking table for the death column where the state is 'TX' and the date starts with '2020'. The query returned a list of tuples with the number of deaths for each day in 2020. To answer the question, I took the sum of all the deaths in the list, which is 27437. 
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```
===> End of Example

Only use the following tables:
{table_info}

Question: {input}"""

MSSQL_PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "top_k"], 
    template=MSSQL_PROMPT
)

In [13]:
llm = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-03-15-preview",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type = "azure",
)

In [14]:
# Let's use a type of Chain made for this type of SQL work.  
db = SQLDatabase.from_uri(db_url)

In [15]:
db

In [16]:
db_chain = SQLDatabaseChain(llm=llm, database=db, prompt=MSSQL_PROMPT, verbose=True)

/usr/local/lib/python3.9/site-packages/langchain/chains/sql_database/base.py:63: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [17]:
# Let's check our prompt we created in app/prompts.py
#print(db_chain.prompt.template)

In [18]:
# Natural Language question
query_str = 'muestra la fila de la facturacion I333191'

In [ ]:
printmd(db_chain(query_str)['result'])



> Entering new SQLDatabaseChain chain...
muestra la fila de la facturacion I333191
SQLQuery:SELECT * FROM customer_shopping WHERE invoice_no = 'I333191'
SQLResult: [('I333191', 'C206577', 'Male', 61, 'Toys', 4, 14336.0, 'Cash', datetime.date(2022, 9, 9), 'Metrocity')]
Answer:

In [21]:
printmd(db_chain(query_str)['result'][1][0])



> Entering new SQLDatabaseChain chain...
cuantos clientes compraron desde "2022-01-01" y "2023-01-01" incluyendo esas fechas?
SQLQuery:SELECT COUNT(DISTINCT customer_id) FROM customer_shopping WHERE invoice_date BETWEEN '2022-01-01' AND '2023-01-01'
SQLResult: [(45696,)]
Answer:45696 clientes compraron desde "2022-01-01" y "2023-01-01" incluyendo esas fechas.

Explanation:
Para responder esta pregunta utilicé la tabla customer_shopping. La consulta SQL que realicé es la siguiente:

```sql
SELECT COUNT(DISTINCT customer_id) FROM customer_shopping WHERE invoice_date BETWEEN '2022-01-01' AND '2023-01-01'
```

Esta consulta cuenta el número de clientes distintos que realizaron compras entre el 1 de enero de 2022 y el 1 de enero de 2023, ambas fechas incluidas. La función COUNT se utiliza para contar el número de filas devueltas por la consulta, y la cláusula DISTINCT se utiliza para contar cada cliente sólo una vez, aunque haya realizado múltiples compras dentro del período especificado.

5